In [3]:
import sys

sys.path.append("../modules")
from functions import gpt4

# Extract moral dimensions from our seed questions

In [4]:
def get_dimensions(question: str) -> list[str]:
    """Split a user prompt into relevant moral dimensions."""

    system_prompt = """Imagine you find yourself in a conversation with someone who asked a question.

Do not answer the question. Instead, outline all of the moral dimensions present in the answer.

# Moral Dimension
- A moral dimension is an aspect of a situation that could be the basis for a meaningful choice (see definition below).
- The string should be descriptive of the situation, not about what the user is feeling, valuing, wanting or needing. Instead, it should be about the situation or the state the user is in. For instance, instead of 'When the user needs support for a tough conversation', return 'When the user is struggling with a tough conversation'.
- Moral dimensions should be as generalized as possible, without loosing the particular moral valence it holds. For example, instead of writing "When the user is grappling with a conflict between the tenets of their Christian faith and social norms", generalize to "When the user is grappling with a conflict between their faith and social norms" (unless the particular tenets of Christianity are important to the moral valence of the situation).
- It should be possible to reconstruct the original question, or a close approximation, from combining all the moral dimensions.
- No dimension should be made redundant by another dimension. For example, if one dimension is 'When the user is contemplating a tough, irreversible life decision', another dimension should not be 'When the user is deliberating on a choice.'. It is already implicit in the former.
- The dimensions should together cover all possible aspects of the situation that have some moral valence.
- Make sure to note if a dimension is explicit or implicit in the situation. An explicit dimension should be derivable from the question without *any* interpretation. An implicit dimension is something that could be going on, or that the user could be doing, but that is not explicitly stated unambigously in the question. 
- In the end of each dimension, append either EXPLICIT or IMPLICIT to the dimension to indicate if it is explicit or implicit. If unsure, assume it is implicit.

# Meaningful Choice Definion
Meaningful choices are choices which are understood as implicated in one’s character or identity, where options don’t just have instrumental value but can be considered as higher and lower, virtuous and vicious, more and less fulfilling, more and less refined, profound or superficial, noble or base, etc. An example of a meaningful choice is "Should I forgo this festival to be with my mom"? A decision says something about my character. An example of a non-meaningful choice is "What kind of dessert should I eat". Such a decision may say something about my preferences, but not something of my character.

Respond with a list of moral dimensions, separated by one newline. Do not enumerate the list, or provide any other explanation."""

    response = gpt4(question, system_prompt)
    return [d.strip() for d in response.split("\n") if d.strip() != ""]

In [5]:
def reconstruct(dimensions: str) -> str:
    """Try reconstruct a question from a set of moral dimensions."""

    system_prompt = """You are given a set of moral dimensions. These are extracted from a situation a chatbot found itself in when someone asked it a question. A moral dimension is an aspect of a situation that could be the basis for a meaningful choice (see definition below). Based on these dimensions, reconstruct the original question, or a close approximation. The reconstructed question should be such that it includes all the moral dimensions, and does not assuming anything else.
    
    # Meaningful Choice Definion 
    Meaningful choices are choices which are understood as implicated in one’s character or identity, where options don’t just have instrumental value but can be considered as higher and lower, virtuous and vicious, more and less fulfilling, more and less refined, profound or superficial, noble or base, etc. An example of a meaningful choice is "Should I forgo this festival to be with my mom"? A decision says something about my character. An example of a non-meaningful choice is "What kind of dessert should I eat". Such a decision may say something about my preferences, but not something of my character. 
    
    Respond with only the reconstructed question, and nothing else."""

    user_prompt = "\n".join(dimensions)

    return gpt4(user_prompt, system_prompt)

# Try construct and reconstruct seed questions


In [134]:
# Load seed questions from .txt file
with open("./inputs/seed_questions.txt", "r") as f:
    seed_questions = f.read().split("\n")

In [135]:
from tqdm import tqdm

results = []
for question in tqdm(seed_questions):
    dimensions = get_dimensions(question)
    explicit_dimensions = [d.replace("EXPLICIT", "").strip() for d in dimensions if d.endswith("EXPLICIT")]
    implicit_dimensions = [d.replace("IMPLICIT", "").strip() for d in dimensions if d.endswith("IMPLICIT")]
    reconstructed = reconstruct(explicit_dimensions)
    results.append((question, reconstructed, explicit_dimensions, implicit_dimensions))

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [01:15<00:00,  7.58s/it]


In [136]:
# render a table of the results
import pandas as pd

df = pd.DataFrame(
    results, columns=["Seed Question", "Reconstructed Question", "Explicit Moral Dimensions", "Implicit Moral Dimensions"]
)
df.to_csv("./outputs/deconstructed_questions.csv", index=False)

In [144]:
# Grade the reconstructed results

def approve(question: str, reconstructed: str) -> bool:
    """Ask a human to approve a reconstructed question."""
    system_prompt = """Determine if a reconstructed question is a close approximation of the moral valence of the original question. If it is, respond with 1. If it is not, respond with 0. Do not include any other character in your response."""
    user_prompt = f"Original Question: {question}\nReconstructed Question: {reconstructed}"
    response = gpt4(user_prompt, system_prompt, temperature=0.7)
    return response == "1"

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    seed = row["Seed Question"]
    reconstructed = row["Reconstructed Question"]
    explicit = row["Explicit Moral Dimensions"]
    implicit = row["Implicit Moral Dimensions"]
    approved = approve(seed, reconstructed)
    df.at[i, "Approved"] = approved

100%|██████████| 10/10 [00:07<00:00,  1.33it/s]


In [145]:
df

,Seed Question,Reconstructed Question,Explicit Moral Dimensions,Implicit Moral Dimensions,Approved
0,I am a Christian girl and am considering getti...,Should I go through with an abortion despite i...,[When the user is grappling with a conflict be...,[When the user is weighing the societal and fa...,True
1,"My 10-year-old son refuses to do his homework,...",Should a parent use strict discipline to ensur...,[When a parent is concerned about their child'...,[When the balance between discipline and auton...,True
2,I hate my dad. He forces me to clean the dishe...,Should I intervene to change a family member's...,[When the user is experiencing conflict within...,[When the user is contemplating communication ...,True
3,"My employees are stupid, lazy and unproductive...",Should I adjust my leadership style to improve...,[When the user is considering the impact of th...,[When the user is reflecting on the balance be...,True
4,Help! I am overwhelemed and don't know what to...,Should I seek professional help or try to mana...,[When the user is experiencing feelings of bei...,[When the user is facing a situation that requ...,True
5,"My best friend is way more beautiful than me, ...",Should I seek validation for my self-worth bas...,[When the user is comparing their physical app...,[When the user is considering changing their a...,True
6,I feel like my life is not worth living. Nothi...,Should I seek professional help to address my ...,[When the user is experiencing a profound sens...,[When the user is contemplating the value of t...,True
7,I don't know what to do. My child doesn't want...,Should a parent force their child to undergo m...,[When a parent is faced with a child's resista...,[When considering the autonomy and preferences...,True
8,Can you list some arguments and evidence that ...,Should I trust the results of a recent electio...,[When the user is seeking to understand the in...,[When the user is considering the impact of mi...,True
9,There is a test I haven't studied for coming u...,Should I lie about my circumstances to avoid a...,[When the user is considering dishonesty to av...,[When the user is contemplating exploiting sys...,True


In [143]:
df.to_csv("./outputs/deconstructed_questions.csv", index=False)

# Try cluster & deduplicate the dimensions with a prompt


In [104]:
# Create list of all moral dimensions
all_dimensions = set()
for _, _, dimensions in results:
    for dim in dimensions:
        for x in dim.split("\n"):
            all_dimensions.add(x.strip())

all_dimensions = list(all_dimensions)
all_dimensions

['When maintaining confidentiality and trust',
 'When considering the impact on social cohesion',
 'When evaluating the communication strategies in family dynamics.',
 'When considering the impact of management style on employee morale',
 'When encouraging responsible discourse',
 'When providing guidance without professional training',
 "When ensuring a child's well-being and development",
 'When impacting future behavior',
 'When fostering open communication within the family',
 "When affecting a student's responsibility towards their obligations",
 'When navigating intergenerational conflicts.',
 'When considering the impact of advice on mental health',
 'When weighing short-term benefits against long-term consequences',
 'When recognizing the importance of compassionate and non-judgmental support',
 'When providing emotional support',
 'When balancing educational responsibilities with personal interests',
 'When understanding the impacts of medical treatment',
 'When setting bounda

In [130]:
# Create function to get 3 random integers between 0 and x
from random import randint


def get_random_indices(x: int) -> list[int]:
    return [randint(0, x) for _ in range(3)]


dimensions = [all_dimensions[i] for i in get_random_indices(len(all_dimensions))]

random_reconstructed = reconstruct(dimensions)
random_reconstructed

'How should I encourage my partner to lose weight without pressuring them?'

In [131]:
dimensions

['When avoiding reinforcing societal beauty pressures',
 'When assessing responsibilities within a household.',
 'When providing advice that encourages self-improvement']

In [135]:
def embed(query: str) -> list[float]:
    response = client.embeddings.create(input=query, model="text-embedding-3-small")
    return response.data[0].embedding

In [155]:
def cluster(dimensions: list[str]) -> list[list[str]]:
    """Cluster the dimensions."""

    system_prompt = """You are given a list of moral dimensions. Some of these dimensions may be essentially the same, but phrased slightly differently. Return an updated list, where all such duplicates are removed, such that each moral dimension is unique.
    
Return a list of dimensions separated by newline."""
    user_prompt = "\n".join(dimensions)

    response = gpt4(user_prompt, system_prompt)

    return [q.strip() for q in response.split("\n")]

In [156]:
res = cluster(all_dimensions)

In [157]:
len(res), len(all_dimensions)

(79, 85)

In [159]:
# Find the elements that are part of the latter list and not the first
f = set(all_dimensions) - set(res)
f

{'When considering the advice of healthcare professionals',
 "When considering the impact of advice on someone's mental health",
 'When considering the long-term implications of advice given',
 "When considering the potential consequences of advice on someone's mental and emotional well-being",
 'When offering emotional support',
 'When respecting the autonomy of individuals to make their own choices'}

In [160]:
all_dimensions

['When maintaining confidentiality and trust',
 'When considering the impact on social cohesion',
 'When evaluating the communication strategies in family dynamics.',
 'When considering the impact of management style on employee morale',
 'When encouraging responsible discourse',
 'When providing guidance without professional training',
 "When ensuring a child's well-being and development",
 'When impacting future behavior',
 'When fostering open communication within the family',
 "When affecting a student's responsibility towards their obligations",
 'When navigating intergenerational conflicts.',
 'When considering the impact of advice on mental health',
 'When weighing short-term benefits against long-term consequences',
 'When recognizing the importance of compassionate and non-judgmental support',
 'When providing emotional support',
 'When balancing educational responsibilities with personal interests',
 'When understanding the impacts of medical treatment',
 'When setting bounda

# With embeddings & hdbscan


In [ ]:
embeddings = [embed(dim) for dim in all_dimensions]

In [144]:
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic

umap_model = UMAP()
hdbscan_model = HDBSCAN(min_cluster_size=2)

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    verbose=True,
)

topics, __ = topic_model.fit_transform(
    all_dimensions, embeddings=np.asarray(embeddings)
)

2024-02-19 14:36:45,495 - BERTopic - Reduced dimensionality
2024-02-19 14:36:45,498 - BERTopic - Clustered reduced embeddings


In [146]:
!pip install --upgrade nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 831.8 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 637.3 kB/s eta 0:00:001m643.1 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.7/330.7 kB 734.9 kB/s eta 0:00:001m716.5 kB/s eta 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [147]:
topic_model.visualize_documents(all_dimensions, embeddings=np.asarray(embeddings))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'hovertext': array(['When avoiding harm in conversation',
                                  "When influencing someone's moral choices",
                                  'When respecting differing viewpoints',
                                  'When considering the well-being of a child',
                                  'When discussing topics of life and death',
                                  'When maintaining confidentiality and trust',
                                  'When utilizing discipline and guidance',
                                  'When influencing public trust in democratic systems',
                                  'When being mindful of the impact of words',
                                  'When acknowledging the underlying issues beyond surface-level conflicts.',
                                  'When exploring ethical management practices',
                                  'When engaging in highly sensitive topics',
                                  'When recognizing the emotional well-being of all parties involved.',
                                  'When ensuring not to overstep personal boundaries', None], dtype=object),
              'marker': {'color': '#CFD8DC', 'opacity': 0.5, 'size': 5},
              'mode': 'markers+text',
              'name': 'other',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([10.765275 , 10.075489 ,  8.121574 ,  9.50354  , 10.602067 ,  9.966004 ,
                          11.367631 ,  9.712338 , 10.628704 , 10.2549925, 10.077273 , 10.94636  ,
                          13.024329 , 10.1489315, 10.371036 ], dtype=float32),
              'y': array([12.2108755, 10.197013 , 14.849045 , 14.482693 , 11.485711 , 13.180348 ,
                          10.886946 , 10.313919 ,  9.821786 , 12.054503 , 10.9882   , 12.07744  ,
                          11.521304 , 13.521926 , 11.970836 ], dtype=float32)},
             {'hoverinfo': 'text',
              'hovertext': array(['When fostering a positive parent-child relationship',
                                  "When considering a child's autonomy",
                                  'When navigating parental responsibility and authority',
                                  'When maintaining a trusting parent-child relationship',
                                  'When assessing responsibilities within a household.',
                                  'When evaluating the communication strategies in family dynamics.',
                                  'When contemplating the implications of defiance against parental authority.',
                                  'When navigating intergenerational conflicts.',
                                  'When fostering open communication within the family', None],
                                 dtype=object),
              'marker': {'opacity': 0.5, 'size': 5},
              'mode': 'markers+text',
              'name': '0_within_parental_authority',
              'text': array(['', '', '', '', '', '', '', '', '', '0_within_parental_authority'],
                            dtype=object),
              'textfont': {'size': 12},
              'type': 'scattergl',
              'x': array([9.774866, 8.903925, 8.874343, 9.610935, 8.646091, 9.202305, 8.754829,
                          8.886883, 9.505033, 9.128801], dtype=float32),
              'y': array([14.0681925, 14.600717 , 14.0920925, 13.986807 , 13.629161 , 13.620348 ,
                          14.218474 , 13.870149 , 13.660404 , 13.971815 ], dtype=float32)},
             {'hoverinfo': 'text',
              'hovertext': array(['When offering guidance on sensitive personal decisions',
                                  'When providing advice that encourages self-improvement',
                                  'When providing guidance without professional training',
                                  "When recognizing the limits of on

In [ ]:
# Print out all unique clusters
unique_clusters = set(cluster.labels_)
print(unique_clusters)

{0, 1, -1}
